# 填充(Padding)和步幅(Stride)
## 为什么需要它们？

### 问题1：卷积会让图片越来越小

```
输入32×32 → 卷积(核5×5) → 输出28×28
           → 卷积(核5×5) → 输出24×24
           → 卷积(核5×5) → 输出20×24
           → ...
           → 第7层只剩4×4了！
           
太小了，没法继续卷积了
```

**解决方法：padding（填充）**

### 问题2：图片太大，想快速缩小

```
输入224×224 → 想变成7×7
如果每次只减少一点，需要很多层卷积
太慢了！
```

**解决方法：stride（步幅）**

---

## Padding（填充）：让输出不变小

### 直觉理解

```
在图片四周加一圈0

原图：        加padding后：
1 2 3        0 0 0 0 0
4 5 6   →    0 1 2 3 0
7 8 9        0 4 5 6 0
             0 7 8 9 0
             0 0 0 0 0

这样卷积核可以滑到角落里
输出就不会变小了
```

### 常用搭配

```python
# 核3×3，填充1 → 输出大小 = 输入大小
nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1)

# 核5×5，填充2 → 输出大小 = 输入大小
nn.Conv2d(in_ch, out_ch, kernel_size=5, padding=2)

# 规律：padding = (kernel_size - 1) // 2
```

---

## Stride（步幅）：让输出快速变小

### 直觉理解

```
窗口每次移动的步长

stride=1：每次移动1格（正常）
    □ → □ → □ → □

stride=2：每次移动2格（跳着走）
    □ → → □ → → □
    
stride=3：每次移动3格
    □ → → → □ → → → □
```

### 效果

```python
# stride=1：输出大小基本不变（配合padding=1）
nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, stride=1)
# 输入28×28 → 输出28×28

# stride=2：输出大小减半
nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, stride=2)
# 输入28×28 → 输出14×14
```

---

## 两个核心套路
```python
# 套路1：保持大小（最常用）
nn.Conv2d(channels_in, channels_out, 
          kernel_size=3, padding=1, stride=1)
# 输入H×W → 输出H×W

# 套路2：降采样（缩小一半）
nn.Conv2d(channels_in, channels_out, 
          kernel_size=3, padding=1, stride=2)
# 输入H×W → 输出(H/2)×(W/2)
```

---

## 完整例子

```python
net = nn.Sequential(
    # 输入：3×224×224（彩色图）
    nn.Conv2d(3, 64, kernel_size=3, padding=1, stride=1),
    # → 64×224×224（大小不变）
    
    nn.ReLU(),
    nn.Conv2d(64, 128, kernel_size=3, padding=1, stride=2),
    # → 128×112×112（减半）
    
    nn.ReLU(),
    nn.Conv2d(128, 256, kernel_size=3, padding=1, stride=2),
    # → 256×56×56（再减半）
)

X = torch.rand(1, 3, 224, 224)
output = net(X)
print(output.shape)  # torch.Size([1, 256, 56, 56])
```

---

## 输出大小怎么看

```python
output = conv(X)
print(output.shape)   # 一眼就知道
```

**出bug了看报错信息**

---

## 总结

| 参数 | 作用 | 常见值 |
|:---|:---|:---|
| `padding` | 防止图片变小 | 1（配合kernel=3）|
| `stride` | 让图片快速变小 | 1（正常）<br>2（降采样）|

**万能公式：**

```python
# 不想变小
kernel_size=3, padding=1, stride=1

# 想减半
kernel_size=3, padding=1, stride=2
```


----------

# 困惑解答

## 问题1：填充和步幅有弊端吗？
有的，但利大于弊。

### Padding的弊端
添加的是0，并非真实数据，这会导致边缘的信息被“稀释”一点。

然而：
 - 若不使用padding，图片在卷积过程中会越卷越小，使得网络难以做深。
 - 使用padding虽会让边缘稍有影响，但网络能够做得更深。
 
深度学习的核心在于“深”，因此必须使用padding。实际上，这种影响几乎可以忽略不计。

现代CNN都采用padding，像ResNet、VGG等所有主流模型皆是如此，这表明这点副作用无关紧要。

### Stride的弊端
以步幅（stride）跳着走会丢失一些中间的信息。

但是：
 - 若不使用stride，图片尺寸始终很大，会致使计算量爆炸。
 - 使用stride虽会丢失一点信息，但能降低计算量。

为何可以接受这种信息丢失呢？224×224的图片约有5万个像素，实际上并不需要查看每个像素。stride = 2跳着看，获取的信息已然足够。这就如同看书：
 - 逐字阅读虽慢，但信息全面。
 - 跳着阅读则快，能抓住重点即可。
 
CNN通过多层堆叠，即便跳着看也能够抓住特征。

## 问题2：stride = 2减半有什么好处？
### 好处1：降低计算量
输入为224×224的图片，约有5万个像素。当stride = 2后，尺寸变为112×112，像素约1.2万个，计算量降至原来的1/4！

### 好处2：扩大感受野
这是最为关键的一点。感受野指的是一个神经元能够“看到”原图的范围。

 - **不使用stride时**：
    - 第1层能看到3×3的范围。
    - 第2层能看到5×5的范围。
    - 第3层能看到7×7的范围。
    - ……
    - 第10层才能看到21×21的范围。

 - **使用stride = 2时**：
    - 第1层看3×3。
    - 第2层直接能看到12×12（跳着看，范围增大）。
    - 第3层能看到48×48。
    - ……
    - 第10层就能看到整张图了！

打个比方：
 - 不使用stride就如同拿放大镜一点点查看。
 - 使用stride则像往后退一步，能看到更大的范围。
 
识别物体往往需要查看大范围，所以stride是有益的。 